In [1]:
from modeling import KNN_Classifier, TokenConversion, TokenExpansion, Word2Vec
from Tokenizer import Tokenizer
from Preprocessing import PreprocessingHashtags,DeleteLowFreqHashtags
import pickle
from keras.preprocessing import image
from gensim.models.word2vec import Word2Vec

import socket, threading;

#KNN 준비
knn = KNN_Classifier.KNN_Classifier()

target = image.load_img('data/test_3.jpg',target_size = (224,224))
knn.load_pickle()

#Word2Vec 준비
model = Word2Vec.load('modeling/model_1')

with open("modeling/tokenized_data.pickle","rb") as fr:
    tokenized = pickle.load(fr)

/Users/pdh/hashtagWebInstagram-hashtag-generator/preprocessing


/Users/pdh/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
def makeReturnString(addr) :
    target = image.load_img(addr,target_size = (224,224))
    tar = knn.img_preprocess(target)
    tar = knn.model(tar)
    knn.show_KNN(tar,5)
    comm_token = knn.get_comm_token(3)
    converted = TokenConversion.TokenConversion(model,comm_token,alpha=0.7,n=10)
    
    returnString = '';
    for data in converted :
        tagList = TokenExpansion.TagExpansion(tokenized,data[0],0.7,0.2)
        for tag in tagList :
            if returnString :
                returnString = returnString + ' ' + '#'+tag;
            else :
                returnString = '#'+tag;
            
    print(returnString)
    return returnString

In [ ]:
def binder(client_socket, addr):
    # 커넥션이 되면 접속 주소가 나온다.
    print('Connected by', addr);
    try:
    # 접속 상태에서는 클라이언트로 부터 받을 데이터를 무한 대기한다.
    # 만약 접속이 끊기게 된다면 except가 발생해서 접속이 끊기게 된다.
        while True:
        # socket의 recv함수는 연결된 소켓으로부터 데이터를 받을 대기하는 함수입니다. 최초 4바이트를 대기합니다.
            data = client_socket.recv(4);
            # 최초 4바이트는 전송할 데이터의 크기이다. 그 크기는 little big 엔디언으로 byte에서 int형식으로 변환한다.
            # C#의 BitConverter는 big엔디언으로 처리된다.
            length = int.from_bytes(data, "little");
            # 다시 데이터를 수신한다.
            data = client_socket.recv(length);
            # 수신된 데이터를 str형식으로 decode한다.
            msg = data.decode();
            # 수신된 메시지를 콘솔에 출력한다.
            print('Received from', addr, msg);
            msg = makeReturnString(msg);
            # 바이너리(byte)형식으로 변환한다.
            data = msg.encode();
            # 바이너리의 데이터 사이즈를 구한다.
            length = len(data);
            # 데이터 사이즈를 little 엔디언 형식으로 byte로 변환한 다음 전송한다.
            client_socket.sendall(length.to_bytes(4, byteorder='little'));
            # 데이터를 클라이언트로 전송한다.
            client_socket.sendall(data);
    except:
        # 접속이 끊기면 except가 발생한다.
        print("except : " , addr);
    finally:
        # 접속이 끊기면 socket 리소스를 닫는다.
        client_socket.close();

# 소켓을 만든다.
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM);
# 소켓 레벨과 데이터 형태를 설정한다.
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1);
# 서버는 복수 ip를 사용하는 pc의 경우는 ip를 지정하고 그렇지 않으면 None이 아닌 ''로 설정한다.
# 포트는 pc내에서 비어있는 포트를 사용한다. cmd에서 netstat -an | find "LISTEN"으로 확인할 수 있다.
server_socket.bind(('', 9999));
# server 설정이 완료되면 listen를 시작한다.
server_socket.listen();
try:
    # 서버는 여러 클라이언트를 상대하기 때문에 무한 루프를 사용한다.
    while True:
        # client로 접속이 발생하면 accept가 발생한다.
        # 그럼 client 소켓과 addr(주소)를 튜플로 받는다.
        client_socket, addr = server_socket.accept();
        th = threading.Thread(target=binder, args = (client_socket,addr));
        # 쓰레드를 이용해서 client 접속 대기를 만들고 다시 accept로 넘어가서 다른 client를 대기한다.
        th.start();
except:
    print("server");
finally:
    # 에러가 발생하면 서버 소켓을 닫는다.
    server_socket.close();


Connected by ('127.0.0.1', 54169)
Received from ('127.0.0.1', 54169) /Users/pdh/hashtagWeb/images/20210516/32825310770835.jpg
['행복_27.jpg', '반려견_7.jpg', '강아지_76.jpg', '개스타그램_85.jpg', '고양이_8.jpg']


/Users/pdh/opt/anaconda3/lib/python3.8/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


#환영맞팔 #멍스타그램 #선팔맞팔 #선팔하면 #선팔하면맞팔 #댕스타그램 #견스타그램 #해요 #좋아요반사 #반려견 #개린이 #하면맞팔
Received from ('127.0.0.1', 54169) 
except :  ('127.0.0.1', 54169)
Connected by ('127.0.0.1', 54199)
Received from ('127.0.0.1', 54199) /Users/pdh/hashtagWeb/images/20210516/32991651702405.jpg
['행복_27.jpg', '반려견_7.jpg', '강아지_76.jpg', '개스타그램_85.jpg', '고양이_8.jpg']
#환영맞팔 #멍스타그램 #선팔맞팔 #선팔하면 #선팔하면맞팔 #댕스타그램 #견스타그램 #해요 #좋아요반사 #반려견 #개린이 #하면맞팔
Received from ('127.0.0.1', 54199) 
except :  ('127.0.0.1', 54199)
Connected by ('127.0.0.1', 54612)
Received from ('127.0.0.1', 54612) /Users/pdh/hashtagWeb/images/20210516/34285811678005.jpg
['꽃_29.jpg', '데이트_37.jpg', '꽃_74.jpg', '얼스타그램_28.jpg', '카페스타그램_51.jpg']
#투어하 #투어하도리 #꽃다발 #꽃바구니 #화분 #꽃집 #플라워레슨 #플로리스트 #용돈꽃다발 #용돈박스 #크로플 #크로플팔이 #개업화분 #개업선물 #개업식
Received from ('127.0.0.1', 54612) 
except :  ('127.0.0.1', 54612)
Connected by ('127.0.0.1', 54618)
Received from ('127.0.0.1', 54618) /Users/pdh/hashtagWeb/images/20210516/34327657770805.jpg
['행복_27.jpg', '반려견_7.jpg', '강아지_76.jpg', '개스